In [6]:

from sklearn.svm import SVC
from sklearn.decomposition import PCA
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
import numpy as np
import joblib
import os
import requests

In [10]:

raw_pipe = {
    "collaborator": [
        {
            "name": "StandardScaler",
            "parameter": []
        },
        {
            "name": "SaveData",
            "parameter": []
        },
        {
            "name": "StandardScaler",
            "parameter": []
        }
    ],
    "global-server": [
        {
            "name": "StandardScaler",
            "parameter": []
        },
        {
            "name": "train_test_split",
            "parameter": []
        },
        {
            "name": "MinMaxScaler",
            "parameter": []
        },
        {
            "name": "SVC",
            "parameter": []
        }
    ]
}


In [26]:
def check_fitted(clf): 
    X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
    y = np.array([1, 1, 2, 2])
    print(clf,hasattr(clf, "predict"))
    return hasattr(clf, "predict")
def parse_global_pipeline(raw_pipe_data,character):
    final_pipeline = []
    param_pipeline = []
    sub_pipeline = []
    pipe = raw_pipe_data[character]
    for idx in range(len(pipe)):
        # print(pipe[idx])
        if(pipe[idx]['name'] != 'SaveData' and pipe[idx]['name'] != 'train_test_split'):
            strp = pipe[idx]['name']+'()'
            if check_fitted(eval(strp)):
                sub_pipeline.append(('model',eval(strp)))
            else:
                sub_pipeline.append((pipe[idx]['name'],eval(strp)))
        elif(pipe[idx]['name'] == 'train_test_split'):
            final_pipeline.append(sub_pipeline)
            sub_pipeline = []
            final_pipeline.append('train_test_split')
        else:
            final_pipeline.append(sub_pipeline)
            final_pipeline.append(pipe[idx]['name'])
            sub_pipeline = []      

    final_pipeline.append(sub_pipeline)
    return final_pipeline


In [28]:
def train_test_split_training(model_pipeline, src_id):
    print(model_pipeline)

parsed_pipeline = parse_global_pipeline(raw_pipe, "global-server")
print(parsed_pipeline)

for sub_pipeline_idx in range(len(parsed_pipeline)):
    sub_pipeline = parsed_pipeline[sub_pipeline_idx]
    
    if(sub_pipeline == 'train_test_split'):
        
        src_id = train_test_split_training(parsed_pipeline[sub_pipeline_idx+1],src_id=1)
        break
    else:
        # train_test_split 之前要儲存資料
        sub_pipeline_param_list = pipe_param_string[parsed_pipeline.index(sub_pipeline)]
        src_id = build_pipeline(dag, src_id, sub_pipeline, param_list=sub_pipeline_param_list, experiment_number=experiment_number)
       





StandardScaler() False
MinMaxScaler() False
SVC() True
[[('StandardScaler', StandardScaler())], 'train_test_split', [('MinMaxScaler', MinMaxScaler()), ('model', SVC())]]
lll
TT
[('MinMaxScaler', MinMaxScaler()), ('model', SVC())]


In [24]:
def parse_param(raw_pipe_data, character):
    final_pipeline_param = []
    param_pipeline = {}
    # sub_pipeline = []
    pipe = raw_pipe_data[character]
    for idx in range(len(pipe)):
        # print('yeeeeeeeeeeeeeeeeeeeeee',pipe[idx])
        if(pipe[idx]['name'] != 'SaveData' and pipe[idx]['parameter']):
            
            # for param in pipe[idx]['parameter'][0]:
            #     newkey = pipe[idx]['name']+'__'+ param
            #     param_pipeline[newkey] = pipe[idx]['parameter'][0][param]


            strp = pipe[idx]['name']+'()'
            if check_fitted(eval(strp)):
                for param in pipe[idx]['parameter'][0]:
                    newkey = 'model__'+ param
                    param_pipeline[newkey] = pipe[idx]['parameter'][0][param]
            else:
                for param in pipe[idx]['parameter'][0]:
                    newkey = pipe[idx]['name']+'__'+ param
                    param_pipeline[newkey] = pipe[idx]['parameter'][0][param]

        elif(pipe[idx]['name'] != 'SaveData' and not pipe[idx]['parameter']):
            # print('NOT', pipe[idx]['name'])
            continue
        else:
            final_pipeline_param.append(param_pipeline)
            param_pipeline = {}
            continue

    # print(final)
    final_pipeline_param.append(param_pipeline)
    return final_pipeline_param

In [17]:
parse_param(raw_pipe, "global-server")

[{}]